# EI ST5 

In [ ]:
# -*- coding: utf-8 -*-

# Python packages
import matplotlib.pyplot
import numpy
import os

# MRG packages
import _env

def myimshow(tab, **kwargs):
    """Customized plot."""

    if 'dpi' in kwargs and kwargs['dpi']:
        dpi = kwargs['dpi']
    else:
        dpi = 100

    # -- create figure
    fig = matplotlib.pyplot.figure(dpi=dpi)
    ax = matplotlib.pyplot.axes()

    if 'title' in kwargs and kwargs['title']:
        title = kwargs['title']
    if 'cmap' in kwargs and kwargs['cmap']:
        cmap = kwargs['cmap']
    else:
        cmap = 'jet'
    #if 'clim' in kwargs and kwargs['clim']:
    #    clim = kwargs['clim']
    if 'vmin' in kwargs and kwargs['vmin']:
        vmin = kwargs['vmin']
    if 'vmax' in kwargs and kwargs['vmax']:
        vmax = kwargs['vmax']

    # -- plot curves
    if 'cmap' in kwargs and kwargs['cmap']:
        matplotlib.pyplot.imshow(tab, cmap=cmap)
    else:
        matplotlib.pyplot.imshow(tab, cmap=cmap)
    if 'title' in kwargs and kwargs['title']:
        matplotlib.pyplot.title(title)
    else:
        matplotlib.pyplot.imshow(tab, cmap=cmap)
    if 'colorbar' in kwargs and kwargs['colorbar']:
        matplotlib.pyplot.colorbar()

#    if 'clim' in kwargs and kwargs['clim']:
#        matplotlib.pyplot.clim(clim)
    if 'vmin' in kwargs and kwargs['vmin']:
        matplotlib.pyplot.clim(vmin, vmax)

    if 'filename' in kwargs and kwargs['filename']:
        output_file = kwargs['filename']
        (root, ext) = os.path.splitext(output_file)
        matplotlib.pyplot.savefig(root + '_plot' + ext, format=ext[1:])
        matplotlib.pyplot.close()
    else:
        matplotlib.pyplot.show()
        matplotlib.pyplot.close()

    matplotlib.pyplot.close(fig)
    return

def _plot_uncontroled_solution(u, chi, freq):
#def _plot_uncontroled_solution(x_plot, y_plot, x, y, u, chi):

    myimshow(numpy.real(u), title='$\operatorname{Re}(u_{0})$ in $\Omega$', colorbar='colorbar', cmap='jet', vmin=-1, vmax=1, filename='fig_u0_re ('+str(freq)+').jpg')
    myimshow(numpy.imag(u), title='$\operatorname{Im}(u_{0})$ in $\Omega$', colorbar='colorbar', cmap='jet', vmin=-1, vmax=1, filename='fig_u0_im('+str(freq)+').jpg')
    myimshow(chi, title='$\chi_{0}$ in $\Omega$', colorbar='colorbar', cmap='jet', vmin=-1, vmax=1, filename='fig_chi0_re.jpg')
    return

def _plot_controled_solution(u, chi, freq):
#def _plot_controled_solution(x_plot, y_plot, x, y, u, chi):

    myimshow(numpy.real(u), title='$\operatorname{Re}(u_{n})$ in $\Omega$', colorbar='colorbar', cmap='jet', vmin=-1, vmax=1, filename='fig_un_re('+str(freq)+').jpg')
    myimshow(numpy.imag(u), title='$\operatorname{Im}(u_{n})$ in $\Omega$', colorbar='colorbar', cmap='jet', vmin=-1, vmax=1, filename='fig_un_im('+str(freq)+').jpg')
    myimshow(chi, title='$\chi_{n}$ in $\Omega$', colorbar='colorbar', cmap='jet', vmin=-1, vmax=1, filename='fig_chin_re.jpg')
    return

def _plot_error(err, freq):

    myimshow(numpy.real(err), title='$\operatorname{Re}(u_{n}-u_{0})$ in $\Omega$', colorbar='colorbar', cmap='jet', vmin = -1, vmax = 1, filename='fig_err_real('+str(freq)+').jpg')
    myimshow(numpy.imag(err), title='$\operatorname{Im}(u_{n}-u_{0})$ in $\Omega$', colorbar='colorbar', cmap='jet', vmin = -1, vmax = 1, filename='fig_err('+str(freq)+').jpg')

    return
def _plot_energy_history(energy):

    matplotlib.pyplot.plot(energy) #, cmap = 'jet')#, vmin = 1e-4, vmax = 1e-0)
    matplotlib.pyplot.title('Energy')
    #matplotlib.pyplot.colorbar()
    #matplotlib.pyplot.show()
    filename = 'fig_energy_real_multirange.jpg'
    matplotlib.pyplot.savefig(filename)
    matplotlib.pyplot.close()

    return


In [ ]:
# -*- coding: utf-8 -*-


# Python packages
import matplotlib.pyplot
import numpy
import scipy
from scipy.optimize import minimize
import scipy.io


def real_to_complex(z):
    return z[0] + 1j * z[1]


def complex_to_real(z):
    return numpy.array([numpy.real(z), numpy.imag(z)])


class Memoize:
    def __init__(self, f):
        self.f = f
        self.memo = {}

    def __call__(self, *args):
        if args not in self.memo:
            self.memo[args] = self.f(*args)
        # .. todo: deepcopy here if returning objects
        return self.memo[args]


def compute_alpha(omega, material):
    """
    .. warning: $w = 2 \pi f$
    w is called circular frequency
    f is called frequency
    """

    # parameters of the material
    gamma_p = 7.0 / 5.0
    rho_0 = 1.2
    c_0 = 340.0
    phi = 0.529  
    sigma = 151429.0  
    alpha_h = 1.37

    # Birch LT
    if material == 'Birch_LT':
        phi = 0.529  # porosity
        sigma = 151429.0  # resitivity
        alpha_h = 1.37  # tortuosity
    if material == 'beton':
        phi = 0.38
        alpha_h = 1
        sigma = 2206.63
    if material == 'Isorel':
        phi = 0.7
        alpha_h = 1.15
        sigma = 142300
    if material == 'nanofiber':
        phi = 0.508
        alpha_h = 2.23
        sigma = 3700
        rho_0 = 2000
    if material == 'liner 1' :
        phi = 0.8
        alpha_h = 0.98
        sigma = 50000
    
    

    # parameters of the geometry
    L = 0.01

    # parameters of the mesh
    resolution = 12  # := number of elements along L

    # parameters of the material (cont.)
    mu_0 = 1.0
    ksi_0 = 1.0 / (c_0 ** 2)
    mu_1 = phi / alpha_h
    ksi_1 = phi * gamma_p / (c_0 ** 2)
    a = sigma * (phi ** 2) * gamma_p / ((c_0 ** 2) * rho_0 * alpha_h)

    ksi_volume = phi * gamma_p / (c_0 ** 2)
    a_volume = sigma * (phi ** 2) * gamma_p / ((c_0 ** 2) * rho_0 * alpha_h)
    mu_volume = phi / alpha_h
    k2_volume = (1.0 / mu_volume) * ((omega ** 2) / (c_0 ** 2)) * (ksi_volume + 1j * a_volume / omega)
    print(k2_volume)

    # parameters of the objective function
    A = 1.0
    B = 1.0

    # defining k, omega and alpha dependant parameters' functions
    @Memoize
    def lambda_0(k, omega):
        if k ** 2 >= (omega ** 2) * ksi_0 / mu_0:
            return numpy.sqrt(k ** 2 - (omega ** 2) * ksi_0 / mu_0)
        else:
            return numpy.sqrt((omega ** 2) * ksi_0 / mu_0 - k ** 2) * 1j

    @Memoize
    def lambda_1(k, omega):
        temp1 = (omega ** 2) * ksi_1 / mu_1
        temp2 = numpy.sqrt((k ** 2 - temp1) ** 2 + (a * omega / mu_1) ** 2)
        real = (1.0 / numpy.sqrt(2.0)) * numpy.sqrt(k ** 2 - temp1 + temp2)
        im = (-1.0 / numpy.sqrt(2.0)) * numpy.sqrt(temp1 - k ** 2 + temp2)
        return complex(real, im)

    @Memoize
    def g(y):
        # ..warning: not validated ***********************
        return 1.0

    @Memoize
    def g_k(k):
        # ..warning: not validated ***********************
        if k == 0:
            return 1.0
        else:
            return 0.0

    @Memoize
    def f(x, k):
        return ((lambda_0(k, omega) * mu_0 - x) * numpy.exp(-lambda_0(k, omega) * L) \
                + (lambda_0(k, omega) * mu_0 + x) * numpy.exp(lambda_0(k, omega) * L))

    @Memoize
    def chi(k, alpha, omega):
        return (g_k(k) * ((lambda_0(k, omega) * mu_0 - lambda_1(k, omega) * mu_1) \
                          / f(lambda_1(k, omega) * mu_1, k) - (lambda_0(k, omega) * mu_0 - alpha) / f(alpha, k)))

    @Memoize
    def eta(k, alpha, omega):
        return (g_k(k) * ((lambda_0(k, omega) * mu_0 + lambda_1(k, omega) * mu_1) \
                          / f(lambda_1(k, omega) * mu_1, k) - (lambda_0(k, omega) * mu_0 + alpha) / f(alpha, k)))

    @Memoize
    def e_k(k, alpha, omega):
        expm = numpy.exp(-2.0 * lambda_0(k, omega) * L)
        expp = numpy.exp(+2.0 * lambda_0(k, omega) * L)

        if k ** 2 >= (omega ** 2) * ksi_0 / mu_0:
            return ((A + B * (numpy.abs(k) ** 2)) \
                    * ( \
                                (1.0 / (2.0 * lambda_0(k, omega))) \
                                * ((numpy.abs(chi(k, alpha, omega)) ** 2) * (1.0 - expm) \
                                   + (numpy.abs(eta(k, alpha, omega)) ** 2) * (expp - 1.0)) \
                                + 2 * L * numpy.real(chi(k, alpha, omega) * numpy.conj(eta(k, alpha, omega)))) \
                    + B * numpy.abs(lambda_0(k, omega)) / 2.0 * ((numpy.abs(chi(k, alpha, omega)) ** 2) * (1.0 - expm) \
                                                                 + (numpy.abs(eta(k, alpha, omega)) ** 2) * (
                                                                             expp - 1.0)) \
                    - 2 * B * (lambda_0(k, omega) ** 2) * L * numpy.real(
                        chi(k, alpha, omega) * numpy.conj(eta(k, alpha, omega))))
        else:
            return ((A + B * (numpy.abs(k) ** 2)) * (L \
                                                     * ((numpy.abs(chi(k, alpha, omega)) ** 2) + (
                                numpy.abs(eta(k, alpha, omega)) ** 2)) \
                                                     + complex(0.0, 1.0) * (1.0 / lambda_0(k, omega)) * numpy.imag(
                        chi(k, alpha, omega) * numpy.conj(eta(k, alpha, omega) \
                                                          * (1.0 - expm))))) + B * L * (
                               numpy.abs(lambda_0(k, omega)) ** 2) \
                   * ((numpy.abs(chi(k, alpha, omega)) ** 2) + (numpy.abs(eta(k, alpha, omega)) ** 2)) \
                   + complex(0.0, 1.0) * B * lambda_0(k, omega) * numpy.imag(
                chi(k, alpha, omega) * numpy.conj(eta(k, alpha, omega) \
                                                  * (1.0 - expm)))

    @Memoize
    def sum_e_k(omega):
        def sum_func(alpha):
            s = 0.0
            for n in range(-resolution, resolution + 1):
                k = n * numpy.pi / L
                s += e_k(k, alpha, omega)
            return s

        return sum_func

    @Memoize
    def alpha(omega):
        alpha_0 = numpy.array(complex(40.0, -40.0))
        temp = real_to_complex(minimize(lambda z: numpy.real(sum_e_k(omega)(real_to_complex(z))), complex_to_real(alpha_0), tol=1e-4).x)
        print(temp, "------", "je suis temp")
        return temp

    @Memoize
    def error(alpha, omega):
        temp = numpy.real(sum_e_k(omega)(alpha))
        return temp

    temp_alpha = alpha(omega)
    temp_error = error(temp_alpha, omega)

    return temp_alpha, temp_error





# Partie 1 : Minimisation à fréquence fixé et multi-fréquences

In [ ]:
# -*- coding: utf-8 -*-


# Python packages
import matplotlib.pyplot
import numpy
import os
import copy


# MRG packages
import _env
import preprocessing
import processing
import postprocessing


def your_optimization_procedure_multirange(freq_range, domain_omega, spacestep, wavenumber_range, f, f_dir, f_neu, f_rob,
                                           beta_pde_range, alpha_pde_range, alpha_dir_range, beta_neu_range, alpha_rob_range, beta_rob_range,
                                           Alpha_range, mu, chi, V_obj, mu1, V_0):
   
    n=0
    (I, J) = numpy.shape(chi)
    for x in range(I):
        for y in range(J):
                if chi[x,y]==1:
                    n+=1
    
    
    k = 0
    (M, N) = numpy.shape(domain_omega)
    numb_iter = 10

    seuil_mu = 1e-6 
    energy = numpy.zeros((numb_iter+1, 1), dtype=numpy.float64)
    while k < numb_iter and mu > seuil_mu:
        print('---- iteration number = ', k)
        # print('1. computing solution of Helmholtz problem, i.e., u')
        u_range = []
        p_range = []
        for f in range(len(freq_range)):
            beta_pde, alpha_pde, alpha_dir, beta_neu, beta_rob, alpha_rob, omega = beta_pde_range[f], alpha_pde_range[
                f], alpha_dir_range[f], beta_neu_range[f], beta_rob_range[f], alpha_rob_range[f], wavenumber_range[f]
            f, f_dir, f_neu, f_rob = f_range[f], f_dir_range[f], f_neu_range[f], f_rob_range[f]

            u = processing.solve_helmholtz(domain_omega, spacestep, omega, f, f_dir, f_neu, f_rob,
                                           beta_pde, alpha_pde, alpha_dir, beta_neu, beta_rob, alpha_rob)

        # print('2. computing solution of adjoint problem, i.e., p')
            f_dir_ = copy.deepcopy(f_dir)
            f_dir_[:, :] = 0
            p = processing.solve_helmholtz(domain_omega, spacestep, omega, -2*numpy.conjugate(u), f_dir_, f_neu, f_rob,
                                           beta_pde, alpha_pde, alpha_dir, beta_neu, beta_rob, alpha_rob)
            u_range.append(u)
            p_range.append(p)
        # print('3. computing objective function, i.e., energy')  # J( chi^(k) )
        ene = compute_objective_function_multirange(
            domain_omega, u_range, spacestep, mu1, V_0)
        energy[k] = ene

        # J'( chi^(k) ) = -Re{alpha*u*p}
        # print('4. computing parametric gradient')
        J_d = compute_J_d_multirange(Alpha_range, u_range, p_range)
        chi_temp = chi.copy()

        while ene >= energy[k] and mu > seuil_mu:
            # Chi^(k+1) = Pl[ Chi^(k) - mu J'( chi^(k) )]
            # print('    a. computing gradient descent')
            chi = compute_gradient_descent(
                domain_omega, chi_temp, J_d, mu)

            #print('    b. computing projected gradient')
            chi = compute_projected(chi, domain_omega, V_obj)

            #print('    c. computing solution of Helmholtz problem, i.e., u')
            u_range = []
            for f, alpha_rob in enumerate(alpha_rob_range):
                alpha_rob = Alpha_range[f] * chi
                alpha_rob_range[f]=alpha_rob
                
                beta_pde, alpha_pde, alpha_dir, beta_neu, beta_rob, omega = beta_pde_range[f], alpha_pde_range[
                    f], alpha_dir_range[f], beta_neu_range[f], beta_rob_range[f], wavenumber_range[f]
                f, f_dir, f_neu, f_rob = f_range[f], f_dir_range[
                    f], f_neu_range[f], f_rob_range[f]

                u_range.append(processing.solve_helmholtz(domain_omega, spacestep, omega, f, f_dir, f_neu, f_rob,
                                                          beta_pde, alpha_pde, alpha_dir, beta_neu, beta_rob, alpha_rob))
                

            #print('    d. computing objective function, i.e., energy (E)')
            ene = compute_objective_function_multirange(
                domain_omega, u_range, spacestep, mu1, V_0)

            if ene <= energy[k]:
                # The step is increased if the energy decreased
                mu = mu*1.1
            else:
                # The step is decreased is the energy increased
                mu = mu / 2
        k += 1

    print('Algorithm executed in {} iterations.'.format(k))
    print('end. computing solution of Helmholtz problem, i.e., u')

    return projected_0_1(chi, n), energy, u_range, J_d

def projected_0_1(chi, N):
    ''' Projeter la valeur de chi dans {0,1}'''
    m,n = numpy.shape(chi)
    liste = []
    for i in range(m):
        for j in range(n):
            liste.append(((i,j),chi[i,j]))
    liste = sorted(liste, key=lambda x:x[1])
    indices=liste[-N:]
    indices_2 = []
    for x,y in indices:
        indices_2.append(x)
    new_chi = []
    for i in range(m):
        for j in range(n):
            if (i,j) in indices_2:
                new_chi.append(1)
            else:
                new_chi.append(0)
    return numpy.array(new_chi).reshape(m,n)


def _set_chi_alea(M, N, x, y, n, alea = False): 
    ''' 
    Initialise le chi_0,  
    n : nombre de case où l'on veut mettre de l'isolant (level 0 : 50 cases sur la frontière) 
    alea : Si on veut placer les n cases aléatoirement ou si on les veut au milieu de la frontière 
    ''' 
    chi = numpy.zeros((M, N), dtype=numpy.float64) 
    val = 1.0 
    if alea : 
        list_choice = numpy.zeros(len(x), dtype = int) 
        for i in range(len(x)) :  
            list_choice[i] = i 
        print("list_choice :", list_choice) 
        for i in range(n) : 
            rand = numpy.random.choice(list_choice) 
            list_choice = numpy.setdiff1d(list_choice, rand) 
            chi[int(y[rand]), int(x[rand])] = val 
    else : 

        for k in range(len(x)//2 - n//2, len(x)//2 + n//2): 

            chi[int(y[k]), int(x[k])] = val 

    return chi 

def _set_chi_n(M, N, x, y, n): 

    """ 
    Initialise chi avec des segments de taille 1/n qui alterne chi=0 et chi=1 
    n : Division demandée (max 50 pour niveau 1) 
    """ 
    val =1.0 
    chi = numpy.zeros((M, N), dtype=numpy.float64)  
    for i in range((n+1)//2): 
        for k in range(2*i*len(x)//n, (2*i+1)*(len(x)//n)): 

            print("k : ", k) 

            chi[int(y[k]), int(x[k])] = val 
    return chi 


def your_compute_objective_function(domain_omega, u, spacestep, mu1, V_0):
    """
    This function compute the objective function:
    J(u,domain_omega)= \int_{domain_omega}||u||^2 + mu1*(Vol(domain_omega)-V_0)
    Parameter:
        domain_omega: Matrix (NxP), it defines the domain and the shape of the
        Robin frontier;
        u: Matrix (NxP), it is the solution of the Helmholtz problem, we are
        computing its energy;
        spacestep: float, it corresponds to the step used to solve the Helmholtz
        equation;
        mu1: float, it is the constant that defines the importance of the volume
        constraint;
        V_0: float, it is a reference volume.
    """
    energy = 0.0
    (M, N) = numpy.shape(domain_omega)
    for i in range(1, M-1):
        for j in range(1, N-1):
            energy += u[i, j]*numpy.conjugate(u[i, j])*(spacestep)**2
    energy = energy + mu1*(M*N*(spacestep**2)-V_0)

    return energy


def compute_objective_function_multirange(domain_omega, u_range, spacestep, mu1, V_0):
    return sum([your_compute_objective_function(domain_omega, u, spacestep, mu1, V_0) for u in u_range])

def compute_J_d(Alpha, u, p):
    return -numpy.real(Alpha*u*p)



def compute_J_d_multirange(Alpha_range, u_range, p_range):
    (M, P) = numpy.shape(u_range[0])
    J_d = numpy.zeros((M, P))
    for f in range(len(Alpha_range)):
        J_d = numpy.add(J_d, compute_J_d(
            Alpha_range[f], u_range[f], p_range[f]))
    return J_d


def compute_gradient_descent(domain_omega, chi, parametric_gradient, mu):
    (M, N) = numpy.shape(domain_omega)
    for x in range(M):
        for y in range(N):
            if domain_omega[x, y] == _env.NODE_ROBIN:
                ct=0
                grad = 0
                if x > 0:
                    ct+=1
                    grad += parametric_gradient[x-1, y]
                if x < M-1:
                    ct+=1
                    grad += parametric_gradient[x+1, y]
                if y > 0:
                    ct+=1
                    grad += parametric_gradient[x, y-1]
                if y < N-1:
                    ct+=1
                    grad += parametric_gradient[x, y+1]
                
                chi[x, y] = chi[x, y] - mu*(grad/ct)
    return chi


def compute_projected(chi, domain, V_obj):

    (M, N) = numpy.shape(domain)
    S = 0
    for i in range(M):
        for j in range(N):
            if domain[i, j] == _env.NODE_ROBIN:
                S = S + 1

    B = chi.copy()
    l = 0
    chi = preprocessing.set2zero(chi, domain)

    V = numpy.sum(numpy.sum(chi)) / S
    debut = -numpy.max(chi)
    fin = numpy.max(chi)
    ecart = fin - debut
    # We use dichotomy to find a constant such that chi^{n+1}=max(0,min(chi^{n}+l,1)) is an element of the admissible space
    while ecart > 10 ** -4:
        # calcul du milieu
        l = (debut + fin) / 2
        for i in range(M):
            for j in range(N):
                chi[i, j] = numpy.maximum(0, numpy.minimum(B[i, j] + l, 1))
        chi = preprocessing.set2zero(chi, domain)
        V = sum(sum(chi)) / S
        if V > V_obj:
            fin = l
        else:
            debut = l
        ecart = fin - debut
        # print('le volume est', V, 'le volume objectif est', V_obj)

    return chi

def _set_chi(M, N, x, y):
    chi = numpy.zeros((M, N), dtype=numpy.float64)
    k_begin = 2 * (len(x) - 1) // 5
    k_end = 5 * (len(x) - 1) // 5
    val = 1.0
    for k in range(k_begin, k_end):
        chi[int(y[k]), int(x[k])] = val
    return chi



if __name__ == '__main__':

    # ----------------------------------------------------------------------
    # -- Fell free to modify the function call in this cell.
    # ----------------------------------------------------------------------
    # -- set parameters of the geometry
    N = 50  # number of points along y-axis

    M = 2*N  # number of points along x-axis
    level = 2  # level of the fractal
    spacestep = 1.0 / N  # mesh size

    
    #freq_range = [175,360,540,700,880] Birch
    #freq_range = [80,250,420,600,770] Nanofiber
    freq_range = [175]
    

    # -- set parameters of the partial differential equation
    sound_speed = 340.0
    wavenumber_range = [2*numpy.pi*frequency /
                        sound_speed for frequency in freq_range]  # wavenumber

    # ----------------------------------------------------------------------
    # -- Do not modify this cell, these are the values that you will be assessed against.
    # ----------------------------------------------------------------------
    # --- set coefficients of the partial differential equation

    beta_pde_range, alpha_pde_range, alpha_dir_range, beta_neu_range, alpha_rob_range, beta_rob_range = [], [], [], [], [], []
    f_range, f_dir_range, f_neu_range, f_rob_range = [], [], [], []
    for _ in range(len(freq_range)):
        beta_pde, alpha_pde, alpha_dir, beta_neu, alpha_rob, beta_rob = preprocessing._set_coefficients_of_pde(
            M, N)
        beta_pde_range.append(beta_pde)
        alpha_pde_range.append(alpha_pde)
        alpha_dir_range.append(alpha_dir)
        beta_neu_range.append(beta_neu)
        alpha_rob_range.append(alpha_rob)
        beta_rob_range.append(beta_rob)

    # -- set right hand sides of the partial differential equation
        f, f_dir, f_neu, f_rob = preprocessing._set_rhs_of_pde(M, N)
        f_range.append(f)
        f_dir_range.append(f_dir)
        f_neu_range.append(f_neu)
        f_rob_range.append(f_rob)

    # -- set geometry of domain
    domain_omega, x, y, _, _ = preprocessing._set_geometry_of_domain(
        M, N, level)

    # ----------------------------------------------------------------------
    # -- Fell free to modify the function call in this cell.
    # ----------------------------------------------------------------------

    # -- define boundary conditions
    # planar wave defined on west
    for f_dir in f_dir_range:
        f_dir[:, :] = 0.0
        f_dir[0, 0:N] = 1.0
    # spherical wave defined on west
    # f_dir[:, :] = 0.0
    # f_dir[int(N/2), 0.0] = 10.0

    for f in range(len(freq_range)):
        alpha_rob_range[f][:, :] = - wavenumber_range[f] * 1j

    # -- define material density matrix
    chi = _set_chi(M, N, x, y)
    
    #chi= _set_chi_alea(M, N, x, y, 20, alea = False)
    #chi= _set_chi_n(M, N, x, y, 10)
    chi = preprocessing.set2zero(chi, domain_omega)

    # -- define absorbing material
    # Alpha = 10.0 - 10.0 * 1j
    #import compute_alpha
    #Alpha = compute_alpha.compute_alpha(...)

    # frequency = [50,100,200,500,1000,2000,5000,10000,20000]
    # Alpha_ = [compute_alpha(2*numpy.pi*f) for f in frequency]
    # Alpha = [A[0] + A[1] * 1j for A in Alpha_]

    Alpha_l_range = [compute_alpha(2*numpy.pi*frequency, 'Isorel')
                     for frequency in freq_range]
    Alpha_range = [Alpha_[0] + Alpha_[1] * 1j for Alpha_ in Alpha_l_range]

    for f in range(len(freq_range)):
        alpha_rob_range[f] = Alpha_range[f] * chi

    # -- set parameters for optimization
    S = 0  # surface of the fractal
    for i in range(0, M):
        for j in range(0, N):
            if domain_omega[i, j] == _env.NODE_ROBIN:
                S += 1
    V_0 = 1  # initial volume of the domain
    V_obj = numpy.sum(numpy.sum(chi)) / S  # constraint on the density
    print(V_obj)
    mu = 5  # initial gradient step
    mu1 = 10**(-9)  # parameter of the volume functional

    # ----------------------------------------------------------------------
    # -- Do not modify this cell, these are the values that you will be assessed against.
    # ----------------------------------------------------------------------
    # -- compute finite difference solution

    chi0 = chi.copy()
    u0_range = []
    for f in range(len(freq_range)):
        beta_pde, alpha_pde, alpha_dir, beta_neu, beta_rob, alpha_rob, wavenumber = beta_pde_range[f], alpha_pde_range[
            f], alpha_dir_range[f], beta_neu_range[f], beta_rob_range[f], alpha_rob_range[f], wavenumber_range[f]
        f, f_dir, f_neu, f_rob = f_range[f], f_dir_range[f], f_neu_range[f], f_rob_range[f]

        u0_range.append(processing.solve_helmholtz(domain_omega, spacestep, wavenumber, f, f_dir, f_neu, f_rob,
                                                   beta_pde, alpha_pde, alpha_dir, beta_neu, beta_rob, alpha_rob))
        

    # ----------------------------------------------------------------------
    # -- Fell free to modify the function call in this cell.
    # ----------------------------------------------------------------------
    # -- compute optimization
    chi, energy, u_range, grad = your_optimization_procedure_multirange(freq_range, domain_omega, spacestep, wavenumber_range, f_range, f_dir_range, f_neu_range, f_rob_range,
                                                                        beta_pde_range, alpha_pde_range, alpha_dir_range, beta_neu_range, alpha_rob_range, beta_rob_range,
                                                                        Alpha_range, mu, chi, V_obj, mu1, V_0)
    chin = chi.copy()
    print("energyyy", energy)

    un_range = [u.copy() for u in u_range]

    # -- plot chi, u, and energy
    for f in range(len(freq_range)):
        u0, un, frequency = u0_range[f], un_range[f], freq_range[f]
        print(u0)
        print('plot figure of frenquency', frequency)
        _plot_uncontroled_solution(u0, chi0, f)
        _plot_controled_solution(un, chin, f)
    #    err = un - u0
    #    _plot_error(err, f)
        _plot_energy_history(energy)
    print('End.')

# Partie 2 : Energie en fonction de la fréquence

In [ ]:
# -*- coding: utf-8 -*-


# Python packages
import matplotlib.pyplot as plt
import numpy
import os

#from time import sleep
#from tqdm.notebook import tdqm
# MRG packages
import _env
import preprocessing
import processing
import postprocessing
from compute_alpha_ref_pierrick import compute_alpha
#import solutions


def g_lim_ref(k):
    return 1


def g_lim_fft_ref(f, omega):  # FFT de g_lim
    if f == 0:
        return 1
    else:
        return 0

def _set_chi_2(M, N, x, y):
    # fonction qui donne chi constant de valeur 1
    chi = numpy.zeros((M, N), dtype=numpy.float64)
    k_begin = 0
    k_end = len(x)-1
    val = 1.0
    for k in range(k_begin, k_end):
        chi[int(y[k]), int(x[k])] = val
    return chi

def your_compute_objective_function(domain_omega, u, spacestep, mu1, V_0):
    """
    This function compute the objective function:
    J(u,domain_omega)= \int_{domain_omega}||u||^2 + mu1*(Vol(domain_omega)-V_0)

    Parameter:
        domain_omega: Matrix (NxP), it defines the domain and the shape of the
        Robin frontier;
        u: Matrix (NxP), it is the solution of the Helmholtz problem, we are
        computing its energy;
        spacestep: float, it corresponds to the step used to solve the Helmholtz
        equation;
        mu1: float, it is the constant that defines the importance of the volume
        constraint;
        V_0: float, it is a reference volume.
    """

    energy = 0.0
    (M, N) = numpy.shape(domain_omega)
    for i in range(M):
        for j in range(N):
            #            if domain_omega[i, j] == _env.NODE_INTERIOR:
            energy += (numpy.absolute(u[i, j])**2)

    return energy*(spacestep**2)

# w=2pi f
# w=2pi/lambda
# On veut 4 points pour un lambda
# n=4lambda= 4*2*pi/w


def plot_energy(f_min, f_max, n, lmaterial):
    """
    Parameters :
    - f_min : valeur de la fréquence minimale
    - f_max : valeur de la fréquence maximale
    - n : nombre de pulsations sur lesquelles calculer l'énergie
    """
    max = {}
    index = {}
    from math import pi
    c = 340
    for material in lmaterial:
        N = 70
        #print("N", N)
        max[material] = 0
        index[material] = 0
        M = 2 * N  # number of points along y-axis
        level = 0  # level of the fractal
        spacestep = 1.0 / N  # mesh size
        print(" on commence les calculs de", material, "-------")
        energy = []
        f_values = numpy.linspace(f_min, f_max, n)
        #print("f_values[-1]", f_values[-1])
        f, f_dir, f_neu, f_rob = preprocessing._set_rhs_of_pde(M, N)
        # -- set right hand sides of the partial differential equation

        # -- set geometry of domain
        domain_omega, x, y, _, _ = preprocessing._set_geometry_of_domain(
            M, N, level)
        f_dir[:, :] = 0.0
        f_dir[0, 0:N] = 1.0
        # spherical wave defined on top
        #f_dir[:, :] = 0.0
        #f_dir[0, int(N/2)] = 10.0

        # -- initialize
        for k in range(n):
            #print("k", k)
            materiau = material
            omega = 2*numpy.pi*f_values[k]
            #print("omega", omega)
            #print(" le f étudié", f_values[k])
            wavenumber = omega / c
            if k == 100:
                print("k =100")
            # ----------------------------------------------------------------------
            # -- Fell free to modify the function call in this cell.
            # ----------------------------------------------------------------------
            # -- set parameters of the geometry
            # n=4lambda= 4*2*pi/w

            # -- set parameters of the partial differential equation
            #kx = -1.0
            #ky = -1.0
            # wavenumber = numpy.sqrt(kx**2 + ky**2)  # wavenumber
            #wavenumber = 10.0
            c = 340

            # ----------------------------------------------------------------------
            # -- Do not modify this cell, these are the values that you will be assessed against.
            # ----------------------------------------------------------------------
            # --- set coefficients of the partial differential equation
            beta_pde, alpha_pde, alpha_dir, beta_neu, alpha_rob, beta_rob = preprocessing._set_coefficients_of_pde(
                M, N)

            # ----------------------------------------------------------------------
            # -- Fell free to modify the function call in this cell.
            # ----------------------------------------------------------------------
            # -- define boundary conditions
            # planar wave defined on top

            alpha_rob[:, :] = - wavenumber * 1j

            # -- define material density matrix

            # fonction qui donne chi constant de valeur 1
            chi =_set_chi_2(M, N, x, y)
            #chi = preprocessing._set_chi(M, N, x, y)
            chi = preprocessing.set2zero(chi, domain_omega)

            # -- define absorbing material
            Alpha = 10.0 - 10.0 * 1j
            # -- this is the function you have written during your project
            #import compute_alpha
            #Alpha = compute_alpha.compute_alpha(...)
            alpha_rob = Alpha * chi

            # -- set parameters for optimization
            S = 0  # surface of the fractal
            for i in range(0, M):
                for j in range(0, N):
                    if domain_omega[i, j] == _env.NODE_ROBIN:
                        S += 1
            V_0 = 1  # initial volume of the domain
            V_obj = numpy.sum(numpy.sum(chi)) / S  # constraint on the density
            mu = 5  # initial gradient step
            mu1 = 10**(-5)  # parameter of the volume functional

            # ----------------------------------------------------------------------
            # -- Boucle qui calcule l'énergie en fonction de la fréquence
            # ----------------------------------------------------------------------

            # plt.show()
            Alpha = compute_alpha(omega, materiau)
            alpha_rob = Alpha * chi
            u = processing.solve_helmholtz(domain_omega, spacestep, wavenumber, f, f_dir,
                                           f_neu, f_rob, beta_pde, alpha_pde, alpha_dir, beta_neu, beta_rob, alpha_rob)
            energy.append(your_compute_objective_function(
                domain_omega, u, spacestep, mu1, V_0))
            if max[material] == 0 or max[material] < energy[-1]:
                max[material] = energy[-1]
                index[material] = omega

        plt.plot(f_values, energy)
        plt.xlabel('Fréquence (Hz)')
        plt.ylabel('Energie')
        plt.title('Énergie en fonction de la fréquence - '+material)
        print(" on étudie", material)
        plt.show()
        plt.savefig(materiau+'.jpg')

        print("max", max, index)


if __name__ == '__main__':

    lmaterial = [
        'fibre de carbone', 'beton armé', 'Liner 1']
    f_min = 50
    f_max = 1000
    # Enfaite pour une frequence donnée et un pas donnée, on doit avoir une augmentation linéaire entre chauqe point...
    n = 500

    plot_energy(f_min, f_max, n, lmaterial)
    from compute_alpha import fonction_final

    print('End.')


# END